In [ ]:
import avstack
import avapi
from tqdm import tqdm
import os
import random
import itertools
import numpy as np

from agents import EgoAgentOnly, EgoDetectionFusion, EgoTrackFusion
from networks import NetworkWithLocalPerception, NetworkWithLocalTracking, NetworkWithDistributedTracking

In [ ]:
data_dir_ego = '/data/spencer/CARLA/ego-lidar/'
data_dir_infra = '/data/spencer/CARLA/multi-agent-v1/'

SM_ego = avapi.carla.CarlaScenesManager(data_dir_ego)
SM_infra = avapi.carla.CarlaScenesManager(data_dir_infra)

In [ ]:
perception = {
    'ego':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
            model='fasterrcnn', dataset='carla', epoch='latest', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(
            model='pointpillars', dataset='carla', epoch='latest', threshold=0.2)
    },
    'agent':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
            model='fasterrcnn', dataset='carla-infrastructure', epoch='latest', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(
            model='pointpillars', dataset='carla-infrastructure', epoch='latest', threshold=0.2)
    },
    'joint':{
        'camera':None,
        'lidar':None
    }
}

In [ ]:
def run_ego_with_network(SD, ego, network, agent_names, n_burnin_save=4, n_frames=10, d_thresh=75):    
    # Run over scenario
    results = []
    for idx, frame in enumerate(tqdm(SD.get_frames(sensor='main_lidar')[2:n_frames])):
        # -- run network 
        if network is not None:
            agent_data = {agent:SD.get_lidar(frame=frame, sensor=agent).transform_to_ground() for agent in agent_names}
            net_out = network(agent_data)
        else:
            net_out = None
    
        # -- pass information to ego vehicle
        ego_pc = SD.get_lidar(frame=frame, sensor='main_lidar')
        ego_out = ego(ego_pc, net_out)
        
        # -- prep ground truths
        objs_care = {}
        objs_dontcare = {}
        for agent in agent_names + ['main_lidar']:
            objects = SD.get_objects(frame=frame, sensor=agent)
            for obj in objects:
                if (obj.ID not in objs_care) and (obj.ID not in objs_dontcare):
                    obj.change_reference(ego_pc.reference, inplace=True)
                    pos = obj.position.change_reference(ego_pc.reference, inplace=False)
                    if pos.norm() <= d_thresh:
                        objs_care[obj.ID] = obj
                    else:
                        objs_dontcare[obj.ID] = obj
                        
        # -- store results
        if idx >= n_burnin_save:
            res = avapi.evaluation.ResultManager(
                idx=frame,
                detections=[track.box3d for track in ego_out],
                truths=[v for k, v in objs_care.items()],
                truths_dontcare=[v for k, v in objs_dontcare.items()],
                metric='3D_IoU',
            )
            results.append(res)
    return results

### Ego-Only

In [ ]:
SD_infra = SM_infra.get_scene_dataset_by_name(SM_infra.splits_scenes['val'][1])

In [ ]:
agent_names = [sensor for sensor in SD_infra.sensor_IDs if 'infra' in sensor.lower() and 'lidar' in sensor.lower()]
ego_local = EgoAgentOnly(perception['ego']['lidar'])
results_local = run_ego_with_network(SD_infra, ego_local, network=None, agent_names=agent_names, n_frames=20)

In [ ]:
idx_view = -10
frame_viz = results_local[idx_view].idx
results_local[idx_view].visualize(lidar=SD_infra.get_lidar(frame=frame_viz, sensor="main_lidar"))

### Simple Fusion At Tracking With Detections

In [ ]:
ego_fusion_1 = EgoDetectionFusion(perception['ego']['lidar'])
agent_names = [sensor for sensor in SD_infra.sensor_IDs if 'infra' in sensor.lower() and 'lidar' in sensor.lower()]
perception_1 = {agent:perception['agent']['lidar'] for agent in agent_names}
network_percep = NetworkWithLocalPerception(perception_1)
results_simple_fusion = run_ego_with_network(SD_infra, ego_fusion_1, network_percep, agent_names, n_frames=20)

In [ ]:
img = SD_infra.get_image(frame_viz, 'main_camera')
img.view()
img = SD_infra.get_image(frame_viz-8, 'CAM_INFRASTRUCTURE_003')
img.view()

In [ ]:
idx_view = -10
frame_viz = results_local[idx_view].idx
results_simple_fusion[idx_view].visualize(lidar=SD_infra.get_lidar(frame=frame_viz, sensor="main_lidar"))

In [ ]:
ego_local = EgoAgentOnly(perception['ego']['lidar'])
results_local = run_ego_with_network(SD, ego_local, network=None, agent_names=[], n_frames=8)

In [ ]:
idx_view = -1
frame_viz = results_local[idx_view].idx
results_local[idx_view].visualize(lidar=SD.get_lidar(frame=frame_viz, sensor="main_lidar"))

### Simple Fusion At Tracking With Tracks

In [ ]:
ego_fusion_2 = FusionAtTracking(perception['ego']['lidar'])
agent_names = [sensor for sensor in SD_infra.sensor_IDs if 'infra' in sensor.lower() and 'lidar' in sensor.lower()]
perception_1 = {agent:perception['agent']['lidar'] for agent in agent_names}
network_tracks = NetworkWithLocalTracking(perception_1)
results_track_fusion = run_ego_with_network(SD_infra, ego_fusion_2, network_tracks, agent_names, n_frames=20)

In [ ]:
idx_view = -1
frame_viz = results_track_fusion[idx_view].idx
results_track_fusion[idx_view].visualize(lidar=SD_infra.get_lidar(frame=frame_viz, sensor="main_lidar"))

### Tracking in a Distributed Network

In [ ]:
net_dist = NetworkWithDistributedTracking(percep_inf, p_share=0.75)
# ego_fuse = EgoDetectionFusion(percep_ego)
ego_track = EgoTrackFusion(percep_ego)

n_frames = 8
results = run_ego_with_network(
    SD=SD,
    ego=ego_track,
    network=net_dist,
    agent_names=agent_names,
    n_frames=n_frames)

In [ ]:
results

In [ ]:
idx_view = -1
frame_viz = results[idx_view].idx
results[idx_view].visualize(lidar=SD.get_lidar(frame=frame_viz, sensor="main_lidar"))

In [ ]:
results

## mAP Experiment

In [ ]:
results_local_all = []
results_fusion_all = []

n_scenes = 4
n_frames = 15

# outer loop is over scenes
scenes = SM_infra.splits_scenes['val'][:n_scenes]

# middle loop is over network types
percep_ego = perception['ego']['lidar']
percep_inf = perception['agent']['lidar']
networks = [('No\nCorrelations', NetworkWithLocalPerception(percep_inf)),
            ('Minor\nCorrelations', NetworkWithDistributedTracking(percep_inf, p_share=0.1)),
            ('Major\nCorrelations', NetworkWithDistributedTracking(percep_inf, p_share=0.75))]

# inner loop is over ego types
egos = [('Local Only', EgoAgentOnly(percep_ego)),
        ('Fusion at Tracking', EgoDetectionFusion(percep_ego)),
        ('Fusion Post Tracking', EgoTrackFusion(percep_ego))]

# --------------------------------------
results_all = []
print('Running over {} egos'.format(len(egos)))
for ego_name, ego in egos:
    ego_res = {}
    print('Running over {} networks'.format(len(networks)))
    for net_name, network in networks:
        # inner most loop is over scenes/frames
        print('Running over {} scenes'.format(len(scenes)))
        ego_res[net_name] = []
        for scene in scenes:
            network.reset()
            ego.reset()
            SD = SM_infra.get_scene_dataset_by_name(scene)
            agent_names = [sensor for sensor in SD.sensor_IDs if 'infra' in sensor.lower() and 'lidar' in sensor.lower()]
            results = run_ego_with_network(
                SD=SD,
                ego=ego,
                network=network,
                agent_names=agent_names,
                n_frames=n_frames)
            ego_res[net_name].extend(results)
    results_all.append((ego_name, ego_res))

In [ ]:
# Convert results to APs
get_ap = avapi.evaluation.metrics.get_ap_from_results
aps_all = [(ego_name, {net_name: get_ap(ego_net_res, by_class=False)[0] for net_name, ego_net_res in ego_res.items()}) for ego_name, ego_res in results_all ]
fig = avapi.evaluation.metrics.barplot_ap(aps_all, reverse_subbars=True, reverse_bars=False, cmap='Accent', color_squeeze=1.3)

out_folder = 'case_study'
os.makedirs(out_folder, exist_ok=True)
fig.savefig(os.path.join(out_folder, 'map_network_study.pdf'), format='pdf', bbox_inches="tight")